In [1]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import gdown
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Define the URL of the CSV file
file_id = '1wmVsSnO3ojRob8sqzvc13h0ui0hMMGEd'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file using gdown
output = 'sampled_data_20.csv'
gdown.download(url, output, quiet=False)

# Read the CSV file into a DataFrame
sampled_data = pd.read_csv(output)

# Display the last few rows of the DataFrame
sampled_data.head()


Downloading...
From (original): https://drive.google.com/uc?id=1wmVsSnO3ojRob8sqzvc13h0ui0hMMGEd
From (redirected): https://drive.google.com/uc?id=1wmVsSnO3ojRob8sqzvc13h0ui0hMMGEd&confirm=t&uuid=4ae84a8f-2724-491f-b287-fd18c9178781
To: /Users/jinluwang/Downloads/Do work here/Challenges/Flight Delay/sampled_data_20.csv
100%|██████████████████████████████████████████| 267M/267M [00:06<00:00, 40.8MB/s]


,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
0,3,5,0,1500-1559,3,4,27,69,"Midwest Airline, Inc.",11965,...,11,Ronald Reagan Washington National,38.852,-77.037,Eppley Airfield,0.00,0.0,0.0,68.0,4.47
1,9,7,0,2200-2259,5,1,7,128,American Airlines Inc.,6740,...,6,Miami International,25.792,-80.286,NONE,0.00,0.0,0.0,94.0,5.82
2,12,1,0,1900-1959,3,5,15,76,Endeavor Air Inc.,5359,...,5,Raleigh-Durham International,35.875,-78.782,Tampa International,0.00,0.0,0.0,50.0,7.83
3,8,3,0,1700-1759,6,2,22,276,United Air Lines Inc.,11012,...,17,Newark Liberty International,40.696,-74.172,San Francisco International,1.69,0.0,0.0,84.0,4.25
4,9,5,0,0800-0859,2,1,3,76,Mesa Airlines Inc.,1378,...,2,Tulsa International,36.198,-95.890,NONE,0.00,0.0,0.0,95.0,6.26


In [3]:
# Check the size of the sampled data
print("Size of the sampled data:", sampled_data.shape)

Size of the sampled data: (1297812, 26)


In [4]:
# Check for missing values in the sampled data
print("Missing Values in Each Column:")
print(sampled_data.isnull().sum())

# Drop rows with missing target variable 'DEP_DEL15'
sampled_data = sampled_data.dropna(subset=['DEP_DEL15'])

# Fill missing values in other columns if needed
sampled_data = sampled_data.fillna(method='ffill')

# Identify categorical features
categorical_features = ['DEP_TIME_BLK', 'CARRIER_NAME', 'DEPARTING_AIRPORT', 'PREVIOUS_AIRPORT']

# One-hot encode categorical variables
data_encoded = pd.get_dummies(sampled_data, columns=categorical_features, drop_first=True)

# Separate features (X) and target (y)
X = data_encoded.drop('DEP_DEL15', axis=1)
y = data_encoded['DEP_DEL15']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Missing Values in Each Column:
MONTH                            0
DAY_OF_WEEK                      0
DEP_DEL15                        0
DEP_TIME_BLK                     0
DISTANCE_GROUP                   0
SEGMENT_NUMBER                   0
CONCURRENT_FLIGHTS               0
NUMBER_OF_SEATS                  0
CARRIER_NAME                     0
AIRPORT_FLIGHTS_MONTH            0
AIRLINE_FLIGHTS_MONTH            0
AIRLINE_AIRPORT_FLIGHTS_MONTH    0
AVG_MONTHLY_PASS_AIRPORT         0
AVG_MONTHLY_PASS_AIRLINE         0
FLT_ATTENDANTS_PER_PASS          0
GROUND_SERV_PER_PASS             0
PLANE_AGE                        0
DEPARTING_AIRPORT                0
LATITUDE                         0
LONGITUDE                        0
PREVIOUS_AIRPORT                 0
PRCP                             0
SNOW                             0
SNWD                             0
TMAX                             0
AWND                             0
dtype: int64


/var/folders/h6/rstm4b2j0g928d95j1p6tr400000gn/T/ipykernel_17026/1958676072.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sampled_data = sampled_data.fillna(method='ffill')


In [5]:
# Train the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.819103647284089
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.99      0.90    210437
           1       0.65      0.10      0.17     49126

    accuracy                           0.82    259563
   macro avg       0.74      0.54      0.53    259563
weighted avg       0.79      0.82      0.76    259563



In [6]:
# Manually specify the features to keep
features_to_keep = [
    'MONTH', 'DAY_OF_WEEK', 'DISTANCE_GROUP', 'SEGMENT_NUMBER', 'CONCURRENT_FLIGHTS', 
    'NUMBER_OF_SEATS', 'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH', 
    'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'AVG_MONTHLY_PASS_AIRPORT', 'AVG_MONTHLY_PASS_AIRLINE', 
    'FLT_ATTENDANTS_PER_PASS', 'GROUND_SERV_PER_PASS', 'PLANE_AGE', 'TMAX', 'AWND'
]

# Add manually encoded categorical features if not included in the list above
for feature in data_encoded.columns:
    if any(cat in feature for cat in categorical_features) and feature not in features_to_keep:
        features_to_keep.append(feature)

# Filter the dataset to keep only the specified features
X = data_encoded[features_to_keep]
y = data_encoded['DEP_DEL15']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Train the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb}")
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

XGBoost Accuracy: 0.8158481755874296
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.99      0.90    210437
           1       0.66      0.06      0.10     49126

    accuracy                           0.82    259563
   macro avg       0.74      0.52      0.50    259563
weighted avg       0.79      0.82      0.75    259563



In [ ]:
# Train the Logistic Regression model
logreg_model = LogisticRegression(random_state=42, max_iter=1000)
logreg_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_logreg = logreg_model.predict(X_test)

# Evaluate the model
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
print(f"Logistic Regression Accuracy: {accuracy_logreg}")
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logreg))

In [ ]:
# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf}")
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
# # # Split the data into training and testing sets
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train a Random Forest model to get feature importance
# rf_model = RandomForestClassifier(random_state=42)
# rf_model.fit(X_train, y_train)

# # Get feature importance
# importance = rf_model.feature_importances_
# feature_names = X.columns

# # Create a DataFrame for visualization
# feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importance})

# # Sort by importance
# feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# # Plot feature importance
# plt.figure(figsize=(10, 6))
# plt.barh(feature_importance_df['feature'], feature_importance_df['importance'])
# plt.xlabel('Feature Importance')
# plt.title('Feature Importance from Random Forest')
# plt.gca().invert_yaxis()
# plt.show()

# # Display the top features
# print("Top 10 Features by Importance:")
# print(feature_importance_df.head(10))


In [ ]:
# # Select top N features (e.g., top 10)
# top_n_features = 10
# top_features = feature_importance_df.head(top_n_features)['feature'].tolist()

# # Create new training and testing sets with only the top features
# X_train_selected = X_train[top_features]
# X_test_selected = X_test[top_features]


In [ ]:
# Train the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train_selected, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test_selected)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Train and evaluate the Logistic Regression model
logreg_model = LogisticRegression(random_state=42, max_iter=1000)
logreg_model.fit(X_train_selected, y_train)
y_pred_logreg = logreg_model.predict(X_test_selected)
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
print(f"Logistic Regression Accuracy: {accuracy_logreg}")
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logreg))

In [ ]:
#Attempts to drop features

# Specify the features to drop
features_to_drop = ['FLT_ATTENDANTS_PER_PASS', 'AVG_MONTHLY_PASS_AIRLINE', 'GROUND_SERV_PER_PASS'] 

# 'LATITUDE','LONGITUDE','NUMBER_OF_SEATS','SEGMENT_NUMBER','DISTANCE_GROUP','PRCP','MONTH','DAY_OF_WEEK','AIRLINE_FLIGHTS_MONTH','AIRPORT_FLIGHTS_MONTH','AIRLINE_AIRPORT_FLIGHTS_MONTH'
# Drop the specified features
data_encoded = data_encoded.drop(columns=features_to_drop)

# Separate features (X) and target (y)
X = data_encoded.drop('DEP_DEL15', axis=1)
y = data_encoded['DEP_DEL15']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score

# Train the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score

# Separate features (X) and target (y)
X = data_encoded.drop('DEP_DEL15', axis=1)
y = data_encoded['DEP_DEL15']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
